## First we unzip the data file

In [1]:
from zipfile import ZipFile
file_name = data_debug.zip
with ZipFile(file_name,'r') as zip:
    zip.extractall()
    print('Done')

ImportError: cannot import name 'Zipfile' from 'zipfile' (c:\users\nikolay dobrev\appdata\local\programs\python\python37\lib\zipfile.py)

## Then we define our utility methods
### First is coming the picture helpers

In [3]:
import matplotlib.pyplot as plt
import pickle
import numpy as np
import torch
import copy
import os.path

In [4]:
def get_segmented_image_7_s_tilde(batch_size,s_tilde):
    
    img_no_downsample = s_tilde
    #img_no_downsample_copy = copy.deepcopy(img_no_downsample)
    #Create a tensor 4D, one dimention for each label [0,3]
    img_3_layer_tensor = img_no_downsample[:,0:3,:,:]
    # take the mean of the indecies > 2
    img_4th_layer_tensor = torch.mean(img_no_downsample[:,4:,:,:], dim=1)
    img_4th_layer_tensor = img_4th_layer_tensor.view(batch_size,1,128,128)
    img_4_layer_tensor = torch.cat([img_3_layer_tensor, img_4th_layer_tensor], dim = 1)
    return img_4_layer_tensor

def get_segmented_image_7(seg_img):
    img_no_downsample = seg_img
    img_no_downsample_copy = copy.deepcopy(img_no_downsample)
    #Create a tensor 4D, one dimention for each label [0,3]
    img_7_lay_tensor = torch.zeros(7, 128, 128)


    for i in range(0,7):
        # first set the value when the index != label to a dummy value = 4
        img_no_downsample_copy[img_no_downsample_copy != i] = 8
        # set the value = 1 when the index = label 
        img_no_downsample_copy[img_no_downsample_copy == i] = 1
        # convert from numpty array to tensor
        img_no_downsample_copy = torch.from_numpy(img_no_downsample_copy)
        #put the tensor in the corresponding index
        img_7_lay_tensor[i, :, :] = img_no_downsample_copy
        # reset the img_no_downsample_copy in order to process the second label 
        img_no_downsample_copy = copy.deepcopy(img_no_downsample)

    #replace the dummy value in the tensor with 0
    img_7_lay_tensor[img_7_lay_tensor == 8] = 0

    # convert the tensot to double instead of unit8 (usigned integer)
    img_7_lay_tensor = img_7_lay_tensor.type(torch.DoubleTensor)
    img_7_lay_tensor = img_7_lay_tensor.resize(1,7,128,128)

    img_7_lay_tensor = img_7_lay_tensor.resize(7,128,128)
    img_7_lay_tensor = img_7_lay_tensor.permute(0,2,1)
        
    return img_7_lay_tensor

def get_downsampled_image_4(img):
    img_no_downsample = img
    img_no_downsample_copy = copy.deepcopy(img_no_downsample)
    # L´ : only use the first four lables. if label > 3, set label = 3
    img_no_downsample[img_no_downsample > 3] = 3
    #Create a tensor 4D, one dimention for each label [0,3]
    img_4_lay_tensor = torch.zeros(4, 128, 128)

    for i in range(0,4):
        # first set the value when the index != label to a dummy value = 4
        img_no_downsample_copy[img_no_downsample_copy != i] = 4
        # set the value = 1 when the index = label 
        img_no_downsample_copy[img_no_downsample_copy == i] = 1
        # convert from numpty array to tensor
        img_no_downsample_copy = torch.from_numpy(img_no_downsample_copy)
        #put the tensor in the corresponding index
        img_4_lay_tensor[i, :, :] = img_no_downsample_copy
        # reset the img_no_downsample_copy in order to process the second label 
        img_no_downsample_copy = copy.deepcopy(img_no_downsample)

    #replace the dummy value in the tensor with 0
    img_4_lay_tensor[img_4_lay_tensor == 4] = 0

    # convert the tensot to double instead of unit8 (usigned integer)
    img_4_lay_tensor = img_4_lay_tensor.type(torch.DoubleTensor)
    img_4_lay_tensor = img_4_lay_tensor.resize(1,4,128,128)
    #segmented_image = segmented_image.view(1,-1,-1,-1)
    # downsampling by 1/8
    img_4_lay_tensor = torch.nn.functional.interpolate(img_4_lay_tensor, scale_factor=(0.0625, 0.0625),  mode='bicubic', align_corners=True)

    img_4_lay_tensor = img_4_lay_tensor.resize(4,8,8)
    img_4_lay_tensor = img_4_lay_tensor.permute(0,2,1)
        
    return img_4_lay_tensor

def get_downsampled_image_4_mS0(batch_size,img):
    img_no_downsample = img

    img_4_lay_tensor = torch.nn.functional.interpolate(img_no_downsample, scale_factor=(0.0625, 0.0625),  mode='bicubic', align_corners=True)

    img_4_lay_tensor = img_4_lay_tensor.resize(batch_size,4,8,8)
        
    return img_4_lay_tensor


def plot_tensor_image(t_img):
    for i in range(len(t_img)):
        segmented_tensor = t_img[i,:,:].resize(8,8)
        plt.imshow(segmented_tensor)
        plt.show()


def plot_tensor_seg_image(t_img):
    for i in range(len(t_img)):
        segmented_tensor = t_img[i,:,:].resize(128,128)
        plt.imshow(segmented_tensor)
        plt.show()

def get_downsampled_batch(batchsize,batch):
    batch_np = batch.cpu().data.numpy()
    batch_down_sampled = torch.ones(batchsize, 4, 8,8)
    for i in range(batchsize):
        batch_down_sampled[i]=get_downsampled_image_4(batch_np[i])
    return batch_down_sampled

def get_segmented_batch(batchsize,batch):
    batch_np = batch.cpu().data.numpy()
    batch_segmented = torch.ones(batchsize, 7, 128,128)
    for i in range(batchsize):
        batch_segmented[i]=get_segmented_image_7(batch_np[i])
    return batch_segmented



### Then we upload the data_loader- data loader is not used in this jupyter notebook but it holds useful signatures

In [5]:
import h5py
import scipy.io
import pickle
from torch.utils.data import Dataset

## Fix here if needed
# language_original_path = os.path.join(os.path.dirname(__file__),'..','data','language_original.mat')
# indeces_path = os.path.join(os.path.dirname(__file__),'..','data','ind.mat')
# segmented_images_raw_path = os.path.join(os.path.dirname(__file__),'..','data','segmented_images.p')
# real_images_raw_path = os.path.join(os.path.dirname(__file__),'..','data','real_images.p')
# h5_file_path = os.path.join(os.path.dirname(__file__),'..','data','G2.h5')
# lang_encoding = os.path.join(os.path.dirname(__file__),'..','data','encode.npy')

def binary_representaiton(val, n_bits):
    binary_str = bin(val)[2:]
    while n_bits > len(binary_str):
            binary_str = '0'+binary_str
    arr =[]
    for i in range(len(binary_str)):
        arr.append(float(binary_str[i]))
    return arr


def create_mask_for_skin_tone(segmented_image):
    mask = copy.deepcopy(segmented_image)
    for i in range(7):
        if(2 == i or 6 == i):
            mask[i == mask] = 1
        else:
            mask[i==mask] = 0
    return mask

def apply_mask(segmented_image,real_image):
    mask = create_mask_for_skin_tone(segmented_image)
    masked = copy.deepcopy(real_image)
    for channel in range(len(real_image)):
        masked[channel] = np.multiply(masked[channel],mask)
    return masked


class FashionData(Dataset):
    def __init__(self,X,y,type_of_data):
        self.X = X[type_of_data]
        self.y = y[type_of_data]
    
    def __getitem__(self, index):
        design_encoding = []
        design_encoding.append(float(self.X['gender'][index]))
        design_encoding.extend(binary_representaiton(self.X['color'][index],5))
        design_encoding.extend(binary_representaiton(self.X['sleeve'][index],3))
        design_encoding.extend(binary_representaiton(self.X['cate_new'][index],5))
        design_encoding.append(self.X['r'][index])
        design_encoding.append(self.X['g'][index])
        design_encoding.append(self.X['b'][index])
        design_encoding.append(self.X['y'][index])
        design_encoding.extend(self.X['encoding'][index])
        design_encoding = np.array(design_encoding)

        return (design_encoding,self.X['down_sampled_images'][index],self.X['segmented_image'][index],self.y[index])        

    def __len__(self):
        return len(self.y)

    

# should we normalize the real_images
def construct_data(segmented_images,real_images,indeces,language,encoded_values):
    X = {}
    y = {}

    X['train'] = {}
    X['train']['gender'] =[]
    X['train']['color'] =[]
    X['train']['sleeve'] =[]
    X['train']['cate_new'] =[]
    X['train']['segmented_image'] = []
    X['train']['down_sampled_images'] = []
    X['train']['description'] = []
    X['train']['encoding'] = []
    X['train']['codeJ'] = []
    X['train']['r'] = []
    X['train']['g'] = []
    X['train']['b'] = []
    X['train']['y'] = []

    X['test'] = {}
    X['test']['gender'] =[]
    X['test']['color'] =[]
    X['test']['sleeve'] =[]
    X['test']['cate_new'] =[]
    X['test']['segmented_image'] = []
    X['test']['down_sampled_images'] = []
    X['test']['description'] = []
    X['test']['encoding'] = []
    X['test']['codeJ'] = []
    X['test']['r'] = []
    X['test']['g'] = []
    X['test']['b'] = []
    X['test']['y'] = []

    y['train'] = []
    y['test'] = []

    # length_to_iterate_train = len(indeces['train_ind'])
    length_to_iterate_train = 10000
    # length_to_iterate_test = len(indeces['test_ind'])
    length_to_iterate_test = 1000


    for i in range(length_to_iterate_train):
        idx = indeces['train_ind'][i][0] - 1
        X['train']['gender'].append(language['gender_'][idx][0])
        X['train']['color'].append(language['color_'][idx][0])
        X['train']['sleeve'].append(language['sleeve_'][idx][0])
        X['train']['cate_new'].append(language['cate_new'][idx][0])
        X['train']['description'].append(str(language['engJ'][idx][0][0]))
        X['train']['encoding'].append(encoded_values[idx])
        # X['train']['segmented_image'].append(segmented_images[idx])
        X['train']['segmented_image'].append(get_segmented_image_7(segmented_images[idx]))  
        X['train']['codeJ'].append(str(language['codeJ'][idx][0][0]))
        skin_tone = apply_mask(np.reshape(segmented_images[idx],(128,128)),real_images[idx])

        r,g,b = np.median(skin_tone[0]), np.median(skin_tone[1]), np.median(skin_tone[2])

        X['train']['r'].append(r)
        X['train']['g'].append(g)
        X['train']['b'].append(b)
        X['train']['y'].append(0.2125*r + 0.7154*g +  0.0721*b)
        #X['train']['down_sampled_images'].append(get_downsampled_image(segmented_images[idx][0]))
        X['train']['down_sampled_images'].append(get_downsampled_image_4(segmented_images[idx]))
        y['train'].append(real_images[idx])

    for i in range(length_to_iterate_test):
        idx = indeces['test_ind'][i][0] - 1
        X['test']['gender'].append(language['gender_'][idx][0])
        X['test']['color'].append(language['color_'][idx][0])
        X['test']['sleeve'].append(language['sleeve_'][idx][0])
        X['test']['cate_new'].append(language['cate_new'][idx][0])
        X['test']['description'].append(str(language['engJ'][idx][0][0]))
        X['test']['encoding'].append(encoded_values[idx])
        X['test']['segmented_image'].append(get_segmented_image_7(segmented_images[idx]))  
        # X['test']['segmented_image'].append(segmented_images[idx])
        X['test']['codeJ'].append(str(language['codeJ'][idx][0][0]))
        skin_tone = apply_mask(np.reshape(segmented_images[idx],(128,128)),real_images[idx])

        r,g,b = np.median(skin_tone[0]), np.median(skin_tone[1]), np.median(skin_tone[2])

        X['test']['r'].append(r)
        X['test']['g'].append(g)
        X['test']['b'].append(b)
        X['test']['y'].append(0.2125*r + 0.7154*g +  0.0721*b)

        X['test']['down_sampled_images'].append(get_downsampled_image_4(segmented_images[idx]))

        y['test'].append(real_images[idx])
    
    return (X,y)

    
def normalize_pictures(real_images):
    for image in real_images:
        for channel in range(len(image)):
            image[channel] = (image[channel] - image[channel].min()) / (image[channel].max()-image[channel].min())
    return real_images


def load_data():

    segmented_images = None
    real_images = None
    print("Check if the serialized data is present")
    # check if the serialized images are present if not create them
    if not(os.path.isfile(segmented_images_raw_path) and os.path.isfile(real_images_raw_path)):
        print("We have to read the h5 file, it will take time")
        with h5py.File(h5_file_path, 'r') as f:   

            # Get the data
            # Segmentated images 1x 128x 128 values from 0 to 6
            segmented_images = list(f['b_'])
            # Real images three channels instad of 0-255 values for a pixel we have normalized values between [-1;1]
            pickle.dump(segmented_images, open(segmented_images_raw_path, 'wb')) 
            real_images = list(f['ih'])
            #normalize the real images
            real_images = normalize_pictures(real_images)
            pickle.dump(real_images, open(real_images_raw_path, 'wb')) 
            print("H5 read and data has been serialized")
    if None == segmented_images:
        infile = open(segmented_images_raw_path,'rb')
        segmented_images = pickle.load(infile)
        infile.close()

    if None == real_images:
        real_images = pickle.load(open(real_images_raw_path,'rb'))
    print("Images has been loaded successfully")
    print("Now reading .mat files")
    # now read language
    lang_org = scipy.io.loadmat(language_original_path)

    # read the indeces as well
    indeces = scipy.io.loadmat(indeces_path)

    print("Everything is loaded now constructing the dictionaries")
    
    encoded_values = np.load(lang_encoding)

    (X,y) = construct_data(segmented_images,real_images,indeces,lang_org, encoded_values)
    print("Data constructed")
    print("Pickle the data")
    handle = open(os.path.join(os.path.dirname(__file__),'..','data','debug_data.pkl'),'wb')
    pickle.dump((X,y), handle, protocol=pickle.HIGHEST_PROTOCOL)
    handle.close()
    
    return (X,y)


NameError: name '__file__' is not defined

### Now the definitions of Discriminators and Generators

In [6]:

import torch.nn as nn


#sizes
human_attributes_size = 18
encoded_description_size = 100
flatten_down_sampled_segmentations_size = 256
gausian_noise_size = 100

design_encoding = human_attributes_size + encoded_description_size + gausian_noise_size # 218



class Generator1(nn.Module):
    def __init__(self):
        super(self).__init__()

        self.G1_Layer2 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=design_encoding,out_channels=1024,kernel_size=4,stride=4,padding=0),
            nn.BatchNorm2d(1024),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=1024,out_channels=512,kernel_size=4,stride=2,padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )

        self.G1_A = nn.Sequential(
            nn.Conv2d(in_channels=4,out_channels=64,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
        )

        self.G1_LastLayers = nn.Sequential(
            nn.ConvTranspose2d(in_channels=(128+512),out_channels=256,kernel_size=4,stride=2,padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=256,out_channels=128,kernel_size=4,stride=2,padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=128,out_channels=64,kernel_size=4,stride=2,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=64,out_channels=7,kernel_size=4,stride=2,padding=1),
            nn.Softmax()
        )

    def forward(self, x_design_desc, x_down_sampled_image):
        x_design = self.G1_Layer2(x_design_desc)
        x_down_sampled = self.G1_A(x_down_sampled_image)
        concatenated = torch.cat((x_design,x_down_sampled),1)
        return self.G1_LastLayers(concatenated)

class Generator2(nn.Module):
    def __init__(self):
        super().__init__()
        self.G2_Layer3 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=design_encoding,out_channels=1024,kernel_size=4,stride=4,padding=0),
            nn.BatchNorm2d(1024),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=1024,out_channels=512,kernel_size=4,stride=2,padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=512,out_channels=256,kernel_size=4,stride=2,padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU()
        )

        self.G2_LayerC = nn.Sequential(
            nn.Conv2d(in_channels=7,out_channels=64,kernel_size=4,stride=2,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=4,stride=2,padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(in_channels=128,out_channels=256,kernel_size=4,stride=2,padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
        )

        self.G2_Layer6 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=512, out_channels=128, kernel_size=4, stride=2, padding=1),            
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=64, out_channels=3, kernel_size=4, stride=2, padding=1),
            nn.Tanh(),
        )
        
    def forward(self, x_design_desc, s_tilde):
        l3 = self.G2_Layer3(x_design_desc)
        lc = self.G2_LayerC(s_tilde)
        concatenated = torch.cat((l3,lc),dim=1)
        return self.G2_Layer6(concatenated)



class Discriminator1(nn.Module):
    def __init__(self):
        super(self).__init__()

        self.D1_Layer1 = nn.Sequential(
            nn.Conv2d(in_channels=7,out_channels=64,kernel_size=4,stride=2,padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=4,stride=2,padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Conv2d(in_channels=128,out_channels=256,kernel_size=4,stride=2,padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
            nn.Conv2d(in_channels=256,out_channels=512,kernel_size=4,stride=2,padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),
        )

        self.D1_condition = nn.Sequential(
            nn.Conv2d(in_channels=4,out_channels=64,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
        )

        self.D1_concatenation = nn.Sequential(
            nn.Conv2d(in_channels=640,out_channels=1024,kernel_size=4,stride=2,padding=1),
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.2),
        )

        self.D1_LastLayers = nn.Sequential(
            nn.Conv2d(in_channels=1142,out_channels=1024,kernel_size=1,stride=1,padding=1),
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.2),
            nn.Conv2d(in_channels=1024,out_channels=1,kernel_size=4,stride=4,padding=0),
            nn.BatchNorm2d(1),
            nn.LeakyReLU(0.2),
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x_segmented_image,x_down_sampled, x_design_encoding):
        layer4 = self.D1_Layer1(x_segmented_image)
        x_down_sampled = self.D1_condition(x_down_sampled)
        
        concatenated = torch.cat((layer4,x_down_sampled),1)
        layer5 = self.D1_concatenation(concatenated)
        d_by_4 = x_design_encoding.repeat(1,16)
        d_by_4 = d_by_4.view(layer5.shape[0],x_design_encoding.shape[1],4,4)
        input_for_layer6 = torch.cat((layer5,d_by_4),1)
        output = self.D1_LastLayers(input_for_layer6)
        output = output.view(x_segmented_image.shape[0],1)
        return self.sigmoid(output)



class Discriminator2(nn.Module):
    def __init__(self):
        super(self).__init__()
        self.D2_Layer3 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),            
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2)            
        )

        self.D2_LayerC = nn.Sequential(
            nn.Conv2d(in_channels=7,out_channels=64,kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),            
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),            
            nn.Conv2d(in_channels=128,out_channels=256,kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2)
            
        )

        self.D2_Layer5 = nn.Sequential(
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),
            nn.Conv2d(in_channels=512,out_channels=1024,kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.2)         
        )

        self.D2_Layer7 = nn.Sequential(
            nn.Conv2d(in_channels=1142,out_channels=1024,kernel_size=1, stride=1, padding=0),
            nn.LeakyReLU(0.2),
            nn.BatchNorm2d(1024),
            nn.Conv2d(in_channels=1024,out_channels=1,kernel_size=4, stride=4, padding=0),
            nn.LeakyReLU(0.2),
            nn.BatchNorm2d(1)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, design_encoding, g2_output, s_tilde):
        l3 = self.D2_Layer3(g2_output)
        lc = self.D2_LayerC(s_tilde)
        concatenated = torch.cat((l3.lc),dim=1)
        l5 = self.D2_Layer5(concatenated)
        l7 = self.D2_Layer7(l5)
        return self.sigmoid(l7)


## Now the training loop

In [ ]:
import data_loader
from torch.utils.data import DataLoader
import os
from torch.autograd import Variable
from IPython.display import display, clear_output


cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda else "cpu")
X, y = None,None

# release
# if os.path.isfile(os.path.join(os.path.dirname(__file__),'..','data','data.pkl')):
#     with open(os.path.join(os.path.dirname(__file__),'..','data','data.pkl')) as handle:
#         X,y = pickle.load(handle)

# debug
loaded_data = None
# if os.path.isfile(os.path.join(os.path.dirname(__file__),'..','data','debug_data_10k.pkl')):
#     with open(os.path.join(os.path.dirname(__file__),'..','data','debug_data_10k.pkl'),'rb') as handle:
if os.path.isfile(os.path.join(os.path.dirname(__file__),'..','data','debug_data.pkl')):
    with open(os.path.join(os.path.dirname(__file__),'..','data','debug_data.pkl'),'rb') as handle:
        loaded_data = pickle.load(handle)
        X,y = loaded_data[0],loaded_data[1]
else:
    X, y = load_data()
training_data = FashionData(X,y,'train')
testing_data = FashionData(X,y,'test')

batch_size = 50

train_loader = DataLoader(training_data, batch_size=batch_size,pin_memory=cuda)
test_loader  = DataLoader(testing_data, batch_size=batch_size, pin_memory=cuda)

flatten_image_size = 128*128

latent_dim_g2 = flatten_image_size + GANs.gausian_noise_size + GANs.human_attributes_size # 16492

G1 = GANs.Generator1()
D1 = GANs.Discriminator1()
if cuda:
    G1.cuda()
    D1.cuda()

loss = torch.nn.BCELoss()
print("Using device:", device)

generator_1_optim = torch.optim.Adam(G1.parameters(), 2e-4, betas=(0.5, 0.999))
discriminator_1_optim = torch.optim.Adam(D1.parameters(), 2e-4, betas=(0.5, 0.999))



# S0: segmented image
# mS0: downsampled segmented image
# d: designed encoding
# dz: {d,z} encoding and noise
# y: real image
#TODO assign data to these variables above
#TODO whether create variables for each step or just once?
#TODO should batch works?
#TODO debug the language encoder function
#TODO encode all data or run encoder during training
#TODO whether need to add condition loss in updating G2?
tmp_img = "tmp_gan_out.png"
discriminator_loss, generator_loss = [], []

num_epochs = 5
for epoch in range(num_epochs):
    batch_d_loss, batch_g_loss = [], []
    
    for i , data in enumerate(train_loader, 0):
        
        d, mS0, S0, label = data
        
        true_label = torch.ones(batch_size, 1).to(device)
        fake_label = torch.zeros(batch_size, 1).to(device)
        
        D1.zero_grad()
        G1.zero_grad()

        
        
        #################### Update D #############################
        # loss 1. real image + real condition -> 1
        x_true_S0 = Variable(S0).to(device,dtype=torch.float)
        x_true_mS0 = Variable(mS0).to(device,dtype=torch.float)
        x_true_d = Variable(d).to(device,dtype=torch.float)        
        output = D1.forward(x_true_S0,x_true_mS0,x_true_d)
        
        error_true = loss(output, true_label) 
        error_true.backward()

        # loss 2. sampled real image + wrong condition -> 0
        # shuffle d     
        # shuffle the true d row wise
        x_notmatch_d = x_true_d[torch.randperm(x_true_d.size()[0])]
        
        x_notmatch_d = Variable(x_notmatch_d).to(device)    
        output = D1.forward(x_true_S0 ,x_true_mS0,x_notmatch_d)

        error_notmatch = loss(output, fake_label) 
        error_notmatch.backward()

        # loss 3. generated fake image + real condition -> 0s
        # z = torch.randn(batch_size, 100, 1, 1,dtype=torch.float64)
        z = torch.randn(batch_size, 100,dtype=torch.float64)
        dz = torch.cat([d, z] , dim=1)
        dz = dz.view((batch_size,dz.shape[1],1,1))
        dz = Variable(dz).to(device,dtype=torch.float)
        x_g_mS0 = Variable(mS0).to(device,dtype=torch.float)

        S_tilde = G1.forward(dz,x_g_mS0)

        x_fake_S = S_tilde
        # x_fake_S = Variable(S_tilde).to(device)
        mS_tilde = down_sample.get_segmented_image_7_s_tilde(batch_size, S_tilde)
        x_fake_mS = down_sample.get_downsampled_image_4_mS0(batch_size, mS_tilde)
        x_fake_mS = Variable(x_fake_mS).to(device,dtype=torch.float)
        x_fake_d = Variable(d).to(device,dtype=torch.float) 
        output = D1.forward(x_fake_S.detach(),x_fake_mS.detach(),x_fake_d.detach())
        

        error_fake = loss(output, fake_label)#log(1-log(g(z)))
        error_fake.backward()
        discriminator_1_optim.step()

            
        
        
        #################### Update G #############################
        
        
        # Step 4. Send fake data through discriminator _again_
        #         propagate the error of the generator and
        #         update G weights.
        output = D1.forward(x_fake_S,x_fake_mS,x_fake_d)
        
        error_generator = loss(output, true_label)
        error_generator.backward()
        generator_1_optim.step()
        
        # batch_d_loss.append((error_true/(error_true + error_fake + error_notmatch)).item())
        batch_d_loss.append((error_true + error_fake + error_notmatch).item())
        batch_g_loss.append(error_generator.item())

    discriminator_loss.append(np.mean(batch_d_loss))
    generator_loss.append(np.mean(batch_g_loss))
    
##################################
    print('Training epoch %d: discriminator_loss = %.5f, generator_loss = %.5f' % (epoch, discriminator_loss[epoch].item(), generator_loss[epoch].item()))

s   
    # Generate data

    with torch.no_grad():
        zsample = torch.randn(100,dtype=torch.float64)
        dsample = []
        dsample.append(float(X['train']['gender'][0]))
        dsample.extend(data_loader.binary_representaiton(X['train']['color'][0],5))
        dsample.extend(data_loader.binary_representaiton(X['train']['sleeve'][0],3))
        dsample.extend(data_loader.binary_representaiton(X['train']['cate_new'][0],5))
        dsample.append(X['train']['r'][0])
        dsample.append(X['train']['g'][0])
        dsample.append(X['train']['b'][0])
        dsample.append(X['train']['y'][0])
        dsample.extend(X['train']['encoding'][0])
        dsample = np.array(dsample)
        dsample = torch.from_numpy(dsample)
        dzsample = torch.cat([dsample,zsample] , dim=0)
        dzsample = dzsample.view((1,dzsample.shape[0],1,1))
        dzsample = Variable(dzsample).to(device,dtype=torch.float)
        mS0_sample = X['train']['down_sampled_images'][0]
        mS0_sample = mS0_sample.view((1,4,8,8))
        mS0_sample = Variable(mS0_sample).to(device,dtype=torch.float)
        S_tilde_sample = G1.forward(dzsample,mS0_sample)
    


plt.plot(range(num_epochs), discriminator_loss)
plt.show()
plt.plot(range(num_epochs), generator_loss)
plt.show()

S_tilde_sample = S_tilde_sample.data.cpu().numpy()
S_tilde_sample =  S_tilde_sample.reshape(7,128,128)
fig = plt.figure()
ax = fig.add_subplot(121)
ax.imshow(S_tilde_sample[1])

S_0_sample = X['train']['segmented_image'][0].data.cpu().numpy()
S_0_sample =S_0_sample.reshape(7,128,128)
ax= fig.add_subplot(122)
ax.imshow(S_0_sample[1])
plt.show()

#######save_image#########


# plt.savefig(tmp_img)
# plt.close(f)
# display(Image(filename=tmp_img))
# clear_output(wait=True)

# os.remove(tmp_img)